In [4]:
import pandas as pd
import numpy as np
import soundfile as sf
import librosa
import os
from utils import *
from config import *
from utils import extract_mfcc

sr = 44100
audio_dir = r'.\input\audio'

In [5]:
score = pd.read_excel(r'.\input\score.xlsx')
data = pd.DataFrame(columns=columns)
data['category'] = score['folder_name']
data['file_name'] = score['file_name']
data['score'] = score['score']
data

,category,file_name,score,1_mean,2_mean,3_mean,4_mean,5_mean,6_mean,7_mean,...,4_diff_2_mean,5_diff_2_mean,6_diff_2_mean,7_diff_2_mean,8_diff_2_mean,9_diff_2_mean,10_diff_2_mean,11_diff_2_mean,12_diff_2_mean,13_diff_2_mean
0,200,001-143-L-11.wav,6.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200,002-127-L-11.wav,6.633333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200,003-154-L-11.wav,6.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200,004-131-L-3.wav,6.700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200,005-2023-04-09_05h17m42s.wav,5.533333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1344,702,98-L-2.wav,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1345,702,99-L-1.wav,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1346,702,99-L-2.wav,4.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1347,702,9-L-1.wav,5.700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
from scipy.stats import kurtosis, skew

for i in range(len(data)):
    try:
        wav_path = os.path.join(audio_dir, str(data.loc[i, 'category']), str(data.loc[i, 'file_name']))
        mfcc = extract_mfcc(wav_path)
        means = np.mean(mfcc, axis=0)
        kurtosis_list = np.apply_along_axis(func1d=kurtosis, axis=0, arr=mfcc)
        skewness_list = np.apply_along_axis(func1d=skew, axis=0, arr=mfcc)
        first_mean, first_var = np.apply_along_axis(func1d=first_order_diff_mean_var, axis=0, arr=mfcc)
        second_mean = np.apply_along_axis(func1d=second_order_diff_mean, axis=0, arr=mfcc)

        data.loc[i, "1_mean":"13_mean"] = means
        data.loc[i, "1_kurtosis":"13_kurtosis"] = kurtosis_list
        data.loc[i, "1_skew":"13_skew"] = skewness_list
        data.loc[i, "1_diff_1_mean":"13_diff_1_mean"] = first_mean
        data.loc[i, "1_diff_1_std":"13_diff_1_std"] = first_var
        data.loc[i, "1_diff_2_mean":"13_diff_2_mean"] = second_mean

    except Exception as e:
        print(f"Failed to load {wav_path}: {str(e)}")
        continue

data

,category,file_name,score,1_mean,2_mean,3_mean,4_mean,5_mean,6_mean,7_mean,...,4_diff_2_mean,5_diff_2_mean,6_diff_2_mean,7_diff_2_mean,8_diff_2_mean,9_diff_2_mean,10_diff_2_mean,11_diff_2_mean,12_diff_2_mean,13_diff_2_mean
0,200,001-143-L-11.wav,6.000000,-351.558807,129.699524,-24.393963,27.436533,-4.836143,21.969589,5.772991,...,-0.004324,0.029115,-0.011319,0.004951,0.006631,-0.019848,-0.013718,-0.013251,-0.013526,-0.001809
1,200,002-127-L-11.wav,6.633333,-349.742737,122.780869,-48.852364,30.297501,-18.720875,7.887349,-12.326442,...,-0.015862,-0.00564,-0.014478,-0.007142,-0.003026,0.011058,0.003039,-0.007403,-0.015382,-0.003262
2,200,003-154-L-11.wav,6.100000,-320.001038,94.369255,-25.467773,38.089897,3.871238,25.670464,0.307353,...,-0.009421,0.015123,-0.011728,0.002192,-0.007242,0.004647,-0.015387,-0.008264,-0.007631,-0.008146
3,200,004-131-L-3.wav,6.700000,-333.587067,127.803688,-32.442688,10.504256,-13.509814,16.713568,-4.463919,...,0.002864,0.040273,0.002317,0.014188,-0.010892,0.005344,-0.010376,-0.000062,-0.002669,-0.004549
4,200,005-2023-04-09_05h17m42s.wav,5.533333,-353.6203,167.027374,14.113789,14.079938,14.309244,6.560044,7.098983,...,0.005915,-0.005486,-0.006482,-0.004301,-0.010414,-0.00861,-0.001221,0.003829,0.001279,0.008912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1344,702,98-L-2.wav,5.000000,-397.155762,138.111328,-32.130901,65.280716,-14.923958,-0.11865,2.971155,...,-0.005902,0.003387,-0.005219,0.005256,-0.005245,0.007665,0.000196,0.001184,-0.00138,-0.003859
1345,702,99-L-1.wav,4.000000,-438.306976,123.592171,-72.126961,30.140499,-45.976196,10.44342,-1.760054,...,-0.007243,0.002603,-0.002008,0.000128,-0.000534,0.001634,0.004382,0.000176,0.001937,-0.002273
1346,702,99-L-2.wav,4.200000,-416.274109,76.505959,-87.691742,59.242271,-54.128895,9.280252,-21.653458,...,-0.003568,0.008413,-0.004464,0.007721,0.003451,0.002184,0.000157,-0.003176,-0.001693,0.001533
1347,702,9-L-1.wav,5.700000,-321.688141,96.273041,23.552652,2.496489,-5.074949,23.720148,-29.09606,...,0.017664,0.000438,0.004941,-0.00171,-0.0047,-0.008255,-0.003897,0.000495,0.002937,0.002551


In [7]:
data.to_excel('./output/78_columns_mfcc_use_laeq.xlsx', index=False, encoding='utf_8_sig')